In [ ]:
!pip install --upgrade transformers accelerate
!pip install bitsandbytes
!pip install trl
!pip install peft
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install datasets --upgrade

In [ ]:
import locale
locale.getpreferredencoding = lambda x: "UTF-8"

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, BitsAndBytesConfig
set_seed(42)

quantization_config = BitsAndBytesConfig(
    load_in8bit=True
)


model_checkpoint = "google/gemma-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map={"":0}, quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
print(model)
print(model.dtype)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
    

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

config = LoraConfig(
    r=LORA_R,
    target_modules=['q_proj', 'v_proj'],
    task_type="CASUAL_LM",
    bias="none",
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, config)

In [ ]:
prompt = "Quote: Imagination is more"
device= "cuda:0"
token_inputs = tokenizer(prompt, return_tensors="pt").to(device)
token_outputs = model.generate(input_ids=token_inputs['input_ids'], max_new_tokens=20)
print(tokenizer.decode(token_outputs[0], skip_special_tokens=True))


Quote: Imagination is more important than knowledge. - Albert Einstein

**Explanation:**

The quote highlights the crucial role of imagination


In [ ]:
from datasets import load_dataset
data = load_dataset("json", data_files="sample-data.json")
#data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

In [ ]:
import transformers
from trl import SFTTrainer

def formatting_func(example):
  text = f"Prompt: {example['data']}\Subquestions: {example['subquestions']}"
  return text

data = data.shuffle().map(
    lambda data_point: tokenizer(
        formatting_func(data_point),
        truncation=True,
        max_length=1700,
        padding="max_length"
    )
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=128 // 1,
        warmup_steps=1,
        num_train_epochs=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="lora-alpaca-13B-context-qa",
        save_total_limit=1,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
trainer.train(resume_from_checkpoint=False)

ModuleNotFoundError: No module named 'trl'

In [ ]:
model.save_pretrained("lora-alpaca-13B-context-qa")

In [ ]:
text = "Prompt: A few players are playing a boardgame. The current state of the game is as follows. The sun bear is named Lily. The canary does not respect the eel. The doctorfish does not hold the same number of points as the eel. And the rules of the game are as follows. Rule1: If the eel has a name whose first letter is the same as the first letter of the sun bear's name, then the eel does not prepare armor for the dog. Rule2: For the eel, if the belief is that the canary does not respect the eel and the doctorfish does not hold the same number of points as the eel, then you can add \"the eel prepares armor for the dog\" to your conclusions. Rule1 is preferred over Rule2. Based on the game state and the rules and preferences, does the eel prepare armor for the dog? Subquestions: "
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Prompt: A few players are playing a boardgame. The current state of the game is as follows. The sun bear is named Lily. The canary does not respect the eel. The doctorfish does not hold the same number of points as the eel. And the rules of the game are as follows. Rule1: If the eel has a name whose first letter is the same as the first letter of the sun bear's name, then the eel does not prepare armor for the dog. Rule2: For the eel, if the belief is that the canary does not respect the eel and the doctorfish does not hold the same number of points as the eel, then you can add "the eel prepares armor for the dog" to your conclusions. Rule1 is preferred over Rule2. Based on the game state and the rules and preferences, does the eel prepare armor for the dog? Subquestions: 1.1. What is the name of the canary in the game? 1.2. What is the relationship between the eel and the doctorfish according to Rule1? 1.3. What is the relationship between the eel and the sun bear according to Rule2? 